In [26]:
import os
import traceback
import logging

In [2]:
simversion = '_v0'
run_art = 1 # 1 True, 0 False
run_spades = 1
run_panta = 1
# pangenome_data = '/data/hoan/amromics/data/ncbi/Kp100random/' # for pangraph, Ragout, multi-CSAR #v3
pangenome_data = '/data/hoan/amromics/data/ncbi/Kptest/' # for pangraph, Ragout, multi-CSAR
simversion = '_'+ pangenome_data.split('/')[-2] + simversion
quast_output = '/data/hoan/amromics/genome-graph/scaffold_output/quastResults' + simversion

In [25]:
print(simversion)

_Kptest_v0


In [4]:
os.system('rm -r ' + pangenome_data + 'fna/')
os.system('mkdir ' + pangenome_data + 'fna/')
os.system('cp -r '+ pangenome_data + '*.fna.gz '+ pangenome_data + 'fna/')
os.system('~/miniconda3/envs/panta/bin/gzip -d ' + pangenome_data + 'fna/*.fna.gz')
os.system('rm ' + pangenome_data + 'fna/g1*')

rm: cannot remove ‘/data/hoan/amromics/data/ncbi/Kptest/fna/’: No such file or directory
rm: cannot remove ‘/data/hoan/amromics/data/ncbi/Kptest/fna/g1*’: No such file or directory


256

### Simualate reads using ART

In [5]:
# https://github.com/scchess/Art/blob/master/art_illumina_README

In [6]:
sim_dir = '/data/hoan/amromics/simulation/'
art_bin = '/data/hoan/amromics/simulation/art_bin/./art_illumina'

In [7]:
ref_data = '/data/hoan/amromics/simulation/references/GCF_000240185.1_ASM24018v2_genomic.fasta'
sim_output = '/data/hoan/amromics/simulation/art_output/paired_dat' + simversion
# sim_output = '/data/hoan/amromics/simulation/art_output/paired_dat_test'

In [8]:
if run_art:
    # os.system(art_bin+' -ss MSv3 -sam -i '+ref_data+' -p -l 250 -f 70 -m 400 -s 60 -o '+sim_output) # _v3
    os.system(art_bin+' -ss HS25 -sam -i '+ref_data+' -p -l 150 -f 70 -m 400 -s 60 -o '+sim_output) # _v0


    ====================ART====================
             ART_Illumina (2008-2016)          
          Q Version 2.5.8 (June 7, 2016)       
     Contact: Weichun Huang <whduke@gmail.com> 
    -------------------------------------------

                  Paired-end sequencing simulation

Total CPU time used: 118.2

The random seed for the run: 1669966048

Parameters used during run
	Read Length:	150
	Genome masking 'N' cutoff frequency: 	1 in 150
	Fold Coverage:            70X
	Mean Fragment Length:     400
	Standard Deviation:       60
	Profile Type:             Combined
	ID Tag:                   

Quality Profile(s)
	First Read:   HiSeq 2500 Length 150 R1 (built-in profile) 
	First Read:   HiSeq 2500 Length 150 R2 (built-in profile) 

Output files

  FASTQ Sequence Files:
	 the 1st reads: /data/hoan/amromics/simulation/art_output/paired_dat_Kptest_v01.fq
	 the 2nd reads: /data/hoan/amromics/simulation/art_output/paired_dat_Kptest_v02.fq

  ALN Alignment Files:
	 the 1st reads: 

### Run SPADES assembly

In [9]:
spades_output = '/data/hoan/amromics/simulation/art_output/' + 'spades_output' + simversion

In [10]:
if run_spades:
    spades_bin ='~/miniconda3/envs/amromics/bin/spades.py'
    os.system(spades_bin+' --isolate -1 '+sim_output+'1.fq -2 '+sim_output+'2.fq -t 50 -o '+spades_output)

Command line: /home/vanhoan310/miniconda3/envs/amromics/bin/spades.py	--isolate	-1	/data/hoan/amromics/simulation/art_output/paired_dat_Kptest_v01.fq	-2	/data/hoan/amromics/simulation/art_output/paired_dat_Kptest_v02.fq	-t	50	-o	/data/hoan/amromics/simulation/art_output/spades_output_Kptest_v0	

System information:
  SPAdes version: 3.15.2
  Python version: 2.7.5
  OS: Linux-3.10.0-1160.15.2.el7.x86_64-x86_64-with-redhat-7.9-Nitrogen

Output dir: /data/hoan/amromics/simulation/art_output/spades_output_Kptest_v0
Mode: ONLY assembling (without read error correction)
Debug mode is turned OFF

Dataset parameters:
  Isolate mode
  Reads:
    Library number: 1, library type: paired-end
      orientation: fr
      left reads: ['/data/hoan/amromics/simulation/art_output/paired_dat_Kptest_v01.fq']
      right reads: ['/data/hoan/amromics/simulation/art_output/paired_dat_Kptest_v02.fq']
      interlaced reads: not specified
      single reads: not specified
      merged reads: not specified
Asse

### Run panta

In [11]:
# move spades ouput to panta dir
print('Pangenome data: ', pangenome_data)
os.system('cp '+spades_output+'/contigs.fasta ' + pangenome_data + 'g1.fna')
os.system('rm ' + pangenome_data + 'g1.fna.gz')
os.system('~/miniconda3/envs/panta/bin/gzip -k ' + pangenome_data + 'g1.fna')

Pangenome data:  /data/hoan/amromics/data/ncbi/Kptest/


rm: cannot remove ‘/data/hoan/amromics/data/ncbi/Kptest/g1.fna.gz’: No such file or directory


0

In [12]:
# Run panta (change output dir)
# https://askubuntu.com/questions/1252439/not-able-to-activate-conda-environment-through-os-system-command-in-python
conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate panta && '
panta_bin =conda_dir+'python /data/hoan/amromics/panta/panta.py'
panta_output = '/data/hoan/amromics/panta/examples/test/output' + simversion
# panta_output = '/data/hoan/amromics/panta/examples/test/output_Kp100' 
print('PANTA OUTPUT: ', panta_output)
if run_panta:
    cmd_panta = panta_bin + ' -p init -a ' +pangenome_data+ '*.fna.gz -o '+panta_output +' -as -s'
    os.system(cmd_panta)
else:
    print("Please RUN PANTA in bash file, here DONOT WORK Because of abPOA")

PANTA OUTPUT:  /data/hoan/amromics/panta/examples/test/output_Kp100_Kptest_v0


08:34:19 INFO : Extract protein -- time taken 0:00:22.592481
08:34:35 INFO : Run CD-HIT with 98% identity -- time taken 0:00:15.680392
08:34:35 INFO : Comparing 8423 sequences with 8423 sequences
08:34:54 INFO : Run BLASTP -- time taken 0:00:19.689999
08:34:55 INFO : Cluster with MCL -- time taken 0:00:00.685828
08:34:55 INFO : Reinflate clusters -- time taken 0:00:00.018224
08:34:58 INFO : Split paralogs iterate 0-- time taken 0:00:00.067625
08:34:58 INFO : Split paralogs iterate 1-- time taken 0:00:00.017016
08:34:58 INFO : Split paralogs iterate 2-- time taken 0:00:00.013034
08:34:58 INFO : Split paralogs iterate 3-- time taken 0:00:00.011593
08:34:58 INFO : Split paralogs iterate 4-- time taken 0:00:00.010299
08:34:58 INFO : Split paralogs iterate 5-- time taken 0:00:00.010213
08:34:58 INFO : Split paralogs -- time taken 0:00:02.503005
08:34:58 INFO : Output gene info -- time taken 0:00:00.081119
08:34:58 INFO : Done -- time taken 0:01:01.353637


### Run pangraph

In [13]:
# # Run pangraph
# %load_ext autoreload
# %autoreload 2
from pangraph import PanGraph

In [14]:
# set parameters
data_dir = panta_output 
incomplete_sample_name = 'g1'
assem_dir = spades_output
fasta_gen = 'partial' # 'all', 'partial'

In [15]:
# assem_dir

'/data/hoan/amromics/simulation/art_output/spades_output_Kptest_v0'

In [16]:
pangraph = PanGraph(sample_info=None, gene_info=None, gene_position=None)

In [17]:
# https://stackoverflow.com/questions/4990718/how-can-i-write-a-try-except-block-that-catches-all-exceptions
try:
    maximum_matching = 'greedy'
    pangraph_output_greedy = spades_output + '/contigs_concat_'+ maximum_matching+ simversion + '.fasta'
    pangraph.run_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_greedy, maximum_matching)
except Exception as e:
    logging.error(traceback.format_exc())
    # Logs the error appropriately. 

Set minimum on number of nucleotides =  10 NUMBER OF COMPUTED CONTIGS: 76
Clip the matrix!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!20
Use union graph: overlap + assembly
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Success! File written


In [18]:
try:
    maximum_matching = 'opt'
    pangraph_output_opt = spades_output + '/contigs_concat_'+ maximum_matching+ simversion + '.fasta'
    pangraph.RERUN_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_opt, maximum_matching)
except Exception as e:
    logging.error(traceback.format_exc())
    # Logs the error appropriately. 

Compute maximum matching
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Success! File written


### Run scaffold methods

In [19]:
### Multi-CSAR
conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate py27 && '
csar_bin = conda_dir + '/data/hoan/amromics/assembly/Multi-CSAR/./multi-csar.php'
scaffold_out_dir = '/data/hoan/amromics/genome-graph/scaffold_output/'

In [20]:
multicsar_output = scaffold_out_dir + 'multicsar' + simversion
os.system(csar_bin +' -t '+pangenome_data+'g1.fna -r '+pangenome_data+'fna/ --nuc -o '+multicsar_output)
# os.system(csar_bin +' -t '+pangenome_data+'g1.fna -r '+pangenome_data+'*.fna --nuc -o '+multicsar_output)

Using reference: GCF_000714675.1_ASM71467v1_genomic.fna


1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "/data/hoan/amromics/genome-graph/scaffold_output/multicsar_Kptest_v0/ref1_out/csar_out/GCF_000714675.1_ASM71467v1_genomic.fna.nuc.ntref" of length 5734568
# construct suffix tree for sequence of length 5734568
# (maximum reference length is 2305843009213693948)
# (maximum query length is 18446744073709551615)
# process 57345 characters per dot
#....................................................................................................
# CONSTRUCTIONTIME /home/vanhoan310/miniconda3/envs/py27/opt/mummer-3.23/mummer /data/hoan/amromics/genome-graph/scaffold_output/multicsar_Kptest_v0/ref1_out/csar_out/GCF_000714675.1_ASM71467v1_genomic.fna.nuc.ntref 2.30
# reading input file "/data/hoan/amromics/data/ncbi/Kptest/g1.fna" of length 5275967
# matching query-file "/data/hoan/amromics/data/ncbi/Kptest/g1.fna"
# against subject-file "/data/hoan/amromics/genome-graph/scaffold_output/multicsar_Kptest_v0/ref

Runtime info:
MUMmer: 9.619 seconds
Main algorithm: 0.005 seconds
Total Time: 9.627 seconds
CSAR is done!
Using reference: GCF_000717515.1_ASM71751v1_genomic.fna


1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "/data/hoan/amromics/genome-graph/scaffold_output/multicsar_Kptest_v0/ref2_out/csar_out/GCF_000717515.1_ASM71751v1_genomic.fna.nuc.ntref" of length 5436788
# construct suffix tree for sequence of length 5436788
# (maximum reference length is 2305843009213693948)
# (maximum query length is 18446744073709551615)
# process 54367 characters per dot
#....................................................................................................
# CONSTRUCTIONTIME /home/vanhoan310/miniconda3/envs/py27/opt/mummer-3.23/mummer /data/hoan/amromics/genome-graph/scaffold_output/multicsar_Kptest_v0/ref2_out/csar_out/GCF_000717515.1_ASM71751v1_genomic.fna.nuc.ntref 2.14
# reading input file "/data/hoan/amromics/data/ncbi/Kptest/g1.fna" of length 5275967
# matching query-file "/data/hoan/amromics/data/ncbi/Kptest/g1.fna"
# against subject-file "/data/hoan/amromics/genome-graph/scaffold_output/multicsar_Kptest_v0/ref

Runtime info:
MUMmer: 9.258 seconds
Main algorithm: 0.005 seconds
Total Time: 9.266 seconds
CSAR is done!
Using reference: GCF_000733255.1_ASM73325v1_genomic.fna


...

1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "/data/hoan/amromics/genome-graph/scaffold_output/multicsar_Kptest_v0/ref7_out/csar_out/GCF_002113865.1_ASM211386v1_genomic.fna.nuc.ntref" of length 6081131
# construct suffix tree for sequence of length 6081131
# (maximum reference length is 2305843009213693948)
# (maximum query length is 18446744073709551615)
# process 60811 characters per dot
#....................................................................................................
# CONSTRUCTIONTIME /home/vanhoan310/miniconda3/envs/py27/opt/mummer-3.23/mummer /data/hoan/amromics/genome-graph/scaffold_output/multicsar_Kptest_v0/ref7_out/csar_out/GCF_002113865.1_ASM211386v1_genomic.fna.nuc.ntref 2.49
# reading input file "/data/hoan/amromics/data/ncbi/Kptest/g1.fna" of length 5275967
# matching query-file "/data/hoan/amromics/data/ncbi/Kptest/g1.fna"
# against subject-file "/data/hoan/amromics/genome-graph/scaffold_output/multicsar_Kptest_v0/r

Runtime info:
MUMmer: 9.771 seconds
Main algorithm: 0.005 seconds
Total Time: 9.78 seconds
CSAR is done!
perfect matching with 184 nodes and 16744 edges
    starting init...done [0.001 secs]. 0 trees
    .
done [0.001 secs]. 0 grows, 0 expands, 0 shrinks
    expands: [0.000 secs], shrinks: [0.000 secs], dual updates: [0.000 secs]
cost = -376.0


+++ Multiple CSAR's result to Blossom +++
The informations : 
reference : 
ref1_out/csar_out/scaffolds.nuc.csar with weight 1
ref2_out/csar_out/scaffolds.nuc.csar with weight 1
ref3_out/csar_out/scaffolds.nuc.csar with weight 1
ref4_out/csar_out/scaffolds.nuc.csar with weight 1
ref5_out/csar_out/scaffolds.nuc.csar with weight 1
ref6_out/csar_out/scaffolds.nuc.csar with weight 1
ref7_out/csar_out/scaffolds.nuc.csar with weight 1
padding weight : 0
outGraph : outGraph.nuc
outGCmap : outGCmap.nuc
# open ref1_out/csar_out/scaffolds.nuc.csar success.
# reading...
# padding car graph...
# merging car graph to result graph...
# now resultGraph size: 166
# open ref2_out/csar_out/scaffolds.nuc.csar success.
# reading...
# padding car graph...
# merging car graph to result graph...
# now resultGraph size: 172
# open ref3_out/csar_out/scaffolds.nuc.csar success.
# reading...
# padding car graph...
# merging car graph to result graph...
# now resultGraph size: 178
# open ref4_out/csar_out/scaffold

Multi-CSAR is DONE!


0

In [21]:
### Ragout
# write receipt file
import glob
ref_files_list = glob.glob(pangenome_data+'fna/*')
n_files = len(ref_files_list)
receipt_file_dir = '/data/hoan/amromics/assembly/Ragout/kp100'+simversion+'.rcp'
f = open(receipt_file_dir, "w")
f.write('.references = ')
for idx in range(n_files-1):
    f.write('r' + str(idx) + ',')
f.write('r'+str(n_files-1)+'\n')
f.write(".target = mg1655\n\n")
for idx in range(n_files):
    f.write('r'+str(idx)+'.fasta =' + ref_files_list[idx] + '\n')
f.write('mg1655.fasta = ' + spades_output+'/contigs.fasta\n')
f.close()

In [22]:
# run ragout
ragout_bin = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate py27 && ragout '
ragout_output = scaffold_out_dir + 'ragout' + simversion
os.system(ragout_bin + receipt_file_dir + ' --outdir ' + ragout_output + ' --refine')
# ragout kp100_v2.rcp --outdir output_Kp100p_v2/ --refine

[08:36:45] INFO: Starting Ragout v2.3
[08:36:45] INFO: Running withs synteny block sizes '[5000, 500, 100]'
[08:36:45] INFO: Running Sibelia with block size 5000
[08:43:00] INFO: Running Sibelia with block size 500
[08:49:18] INFO: Running Sibelia with block size 100
[08:55:35] INFO: Inferring phylogeny from synteny blocks data
[08:55:35] INFO: Reading scaffold_output/ragout_Kptest_v0/sibelia-workdir/100/blocks_coords.txt
[08:55:35] INFO: "r4" synteny blocks coverage: 95.14%
[08:55:35] INFO: "r5" synteny blocks coverage: 97.22%
[08:55:35] INFO: "r6" synteny blocks coverage: 97.28%
[08:55:35] INFO: "r0" synteny blocks coverage: 98.24%
[08:55:35] INFO: "r1" synteny blocks coverage: 99.6%
[08:55:35] INFO: "r2" synteny blocks coverage: 96.28%
[08:55:35] INFO: "r3" synteny blocks coverage: 96.74%
[08:55:35] INFO: "mg1655" synteny blocks coverage: 97.78%
[08:55:36] INFO: Inferred tree: (('r0' : 5.1875, 'r1' : 2.8125) : 9.296875, (('mg1655' : 28.375, 'r6' : 1e-06) : 8.65625, ('r3' : 28.46875,

DDFJKJFJSJFDSJFLKSJFLKJSDKFJSLKJFLKJFLKJFSJFLDSJFLKSJDLKJFLJ----------------------------------


	Building FM-index
	Overapping
	Kmer size is set to 77
[08:55:38] INFO: Refining with assembly graph
[08:55:38] INFO: Generating FASTA output
[08:55:38] INFO: Assembly statistics:

	Scaffolds:		5
	Used fragments:		61
	Scaffolds length:	5502669

	Unplaced fragments:	46
	Unplaced length:	14990 (0.28%)
	Introduced Ns length:	240319 (4.37%)

	Fragments N50:		203837
	Assembly N50:		1739291

[08:55:38] INFO: Done!


0

### Quast 

In [23]:
ragout_output

'scaffold_output/ragout_Kptest_v0'

In [24]:
if True:
    quast_bin = 'python /data/hoan/amromics/spades_quast/quast-5.2.0/quast.py '
    ref_genome = '/data/hoan/amromics/simulation/references/GCF_000240185.1_ASM24018v2_genomic.fasta'
    spades_output_fasta = spades_output+'/contigs.fasta'
    multicsar_output_fasta = multicsar_output +'/multi-csar.nuc.out.fna'
    ragout_output_fasta = ragout_output + '/mg1655_scaffolds.fasta'
    # multicsar_output = '/data/hoan/amromics/genome-graph/scaffold_output/multicsar/multi-csar.nuc.out.fna '
    # ragout_output = '/data/hoan/amromics/assembly/Ragout/output_Kp100p_v3/mg1655_scaffolds.fasta '
    os.system(quast_bin + pangraph_output_opt+' '+ pangraph_output_greedy+' '+ multicsar_output_fasta+' '+ragout_output_fasta+' '+spades_output_fasta+' '+
              '-l "Pangraph_OPT, Pangraph_Greedy, Multi-CSAR, Ragout, SPADES" '+ '-r '+ref_genome+' -o '+ quast_output+' --silent --extensive-mis-size 5000 --local-mis-size 3000')

/data/hoan/amromics/spades_quast/quast-5.2.0/quast.py /data/hoan/amromics/simulation/art_output/spades_output_Kptest_v0/contigs_concat_opt_Kptest_v0.fasta /data/hoan/amromics/simulation/art_output/spades_output_Kptest_v0/contigs_concat_greedy_Kptest_v0.fasta scaffold_output/multicsar_Kptest_v0/multi-csar.nuc.out.fna scaffold_output/ragout_Kptest_v0/mg1655_scaffolds.fasta /data/hoan/amromics/simulation/art_output/spades_output_Kptest_v0/contigs.fasta -l Pangraph_OPT, Pangraph_Greedy, Multi-CSAR, Ragout, SPADES -r /data/hoan/amromics/simulation/references/GCF_000240185.1_ASM24018v2_genomic.fasta -o /data/hoan/amromics/genome-graph/scaffold_output/quastResults_Kptest_v0 --silent --extensive-mis-size 5000 --local-mis-size 3000


System information:
  OS: Linux-3.10.0-1160.15.2.el7.x86_64-x86_64-with-redhat-7.9-Nitrogen (linux_64)
  Python version: 2.7.5
  CPUs number: 72

Started: 2022-12-02 08:55:38

Logging to /data/hoan/amromics/genome-graph/scaffold_output/quastResults_Kptest_v0/quast.

Unable to parse the pattern
Unable to parse the pattern
Unable to parse the pattern


Done.

2022-12-02 08:55:48
Running Contig analyzer...
Done.

2022-12-02 08:55:54
Running NA-NGA calculation...
Done.

2022-12-02 08:55:56
Running Genome analyzer...
  NOTICE: No file with genomic features were provided. Use the --features option if you want to specify it.

  NOTICE: No file with operons were provided. Use the -O option if you want to specify it.
Done.

NOTICE: Genes are not predicted by default. Use --gene-finding or --glimmer option to enable it.

2022-12-02 08:56:01
Creating large visual summaries...
This may take a while: press Ctrl-C to skip this step..
  1 of 2: Creating PDF with all tables and plots...
  2 of 2: Creating Icarus viewers...
Done

2022-12-02 08:56:05
RESULTS:
  Text versions of total report are saved to /data/hoan/amromics/genome-graph/scaffold_output/quastResults_Kptest_v0/report.txt, report.tsv, and report.tex
  Text versions of transposed total report are saved to /data/hoan/amromics/genome-graph/scaffold_output/quastResults_Kptest_v0/transposed_